In [89]:
import pandas as pd
from datetime import date, datetime
import re
import os
#Carregando arquivo excel em formato dataframe
dataframe_original = pd.read_excel('Amostras.xlsx', engine='openpyxl') 

dataframe_original.head(5)

,CD_EVOLUCAO,NR_ATENDIMENTO,NM_USUARIO,DS_EVOLUCAO,VALIDACAO
0,7560322,818153,DIANA.SOUSA,ANOTACAO ???DE ENFERMAGEM - NOTURNO PACIENTE E...,NÃO
1,7560345,818153,LUCIANA.FEITOSA,EVOLUCAO DE FISIOTERAPIANOITE\n EM IH POR:- IT...,SIM
2,7561867,818153,AMANDA.NUNES,EVOLUCAO DE ENFERMAGEM - UTI CLINICA I - DIURN...,SIM
3,7561917,818153,KAROLYNE.FREIRE,EVOLUCAO DE FISIOTERAPIAMANHA\n EM IH POR:- IT...,SIM
4,7562674,818153,MARTILIANA.LIMA,PLANTAO MTEM INTERNACAO NA UTI CLINICA I POR S...,NÃO


In [90]:
#Separando apenas as colunas 'DS_EVOLUCAO' e 'VALIDACAO'
dataframe_formatado = dataframe_original[['DS_EVOLUCAO','VALIDACAO']].copy()
#Converto os valores da coluna VALIDACAO SIM para 1 e NÃO para 0
dataframe_formatado.loc[dataframe_formatado['VALIDACAO'] == "SIM", 'VALIDACAO'] = 1
dataframe_formatado.loc[dataframe_formatado['VALIDACAO'] == "NÃO", 'VALIDACAO'] = 0
#Removo todos os caracteres '?'
dataframe_formatado['DS_EVOLUCAO'] = dataframe_formatado['DS_EVOLUCAO'].str.replace('?','',regex=False) #

dataframe_formatado.head(5)

,DS_EVOLUCAO,VALIDACAO
0,ANOTACAO DE ENFERMAGEM - NOTURNO PACIENTE EVOL...,0
1,EVOLUCAO DE FISIOTERAPIANOITE\n EM IH POR:- IT...,1
2,EVOLUCAO DE ENFERMAGEM - UTI CLINICA I - DIURN...,1
3,EVOLUCAO DE FISIOTERAPIAMANHA\n EM IH POR:- IT...,1
4,PLANTAO MTEM INTERNACAO NA UTI CLINICA I POR S...,0


In [108]:
dicionario_termos = [
    "ASPIROU","ASPIREI","ASPIRO","ASPIRADO","ASPIRAR",
    "ASPIRACAO","SECRECAO","TRACHCARE","VAS","TRACH","CARE",
    "VIAS SUPERIORES","AEREAS","SECRECAO","ASP","SISTEMA FECHADO",
    "VENTILACAO MECANICA"
]

negativos = [
    "EUPNEICO", "AR AMBIENTE","EUPNEICA",
    "EUPENEICA", "EUPENEICO", "EUPONEICO"
]

stopwords = ["EM", "DE"]

pattern_stopwords = r"\b(" + "|".join([re.escape(i) for i in stopwords]) + r")\b"
pattern_positivo = r"\b(" + "|".join([re.escape(i) for i in dicionario_termos]) + r")\b"
pattern_negativo = "|".join([re.escape(i) for i in negativos])


In [ ]:
#Função para extrarir X caracteres pre e Y caracteres pos a cada termo encontrado na evolução
def extrair_contexto_caracteres(texto, regex=pattern_positivo, pre=30, pos=30):
    if not isinstance(texto, str):
        return []
    
    resultados = []
    for m in re.finditer(regex, texto, flags=re.IGNORECASE):
        start, end = m.span()
        inicio = max(0, start - pre)      # X caracteres antes
        fim = min(len(texto), end + pos)  # Y caracteres depois
        resultados.append(texto[inicio:fim])
    
    return resultados

In [ ]:
#Função para extrair X palavras pré e Y palavras pós a cada termo encontrado na evolução
def extrair_contexto_por_palavras(texto, regex=pattern_positivo, pre=5, pos=5):
    if not isinstance(texto, str):
        return []
    
    # padroniza para maiúsculas
    texto_up = texto.upper()
    # separa em palavras
    palavras = texto_up.split()
    
    resultados = []

    # percorre palavras
    i = 0
    while i < len(palavras):
        # verifica se o termo positivo bate a partir da posição i
        for p in dicionario_termos:
            termo_palavras = p.upper().split()
            tamanho = len(termo_palavras)
            if palavras[i:i+tamanho] == termo_palavras:
                # pega 'pre' palavras antes e 'pos' palavras depois
                inicio = max(0, i - pre)
                fim = min(len(palavras), i + tamanho + pos)
                contexto = " ".join(palavras[inicio:fim])
                resultados.append(contexto)
                i += tamanho - 1  # pula as palavras do termo encontrado
                break
        i += 1

    return resultados

In [132]:
#df_filtrado_termo_positivo = dataframe_formatado[dataframe_formatado['DS_EVOLUCAO'].str.contains(pattern_positivo, regex=True)]
dataframe_formatado['DS_CONTEXTO'] = dataframe_formatado['DS_EVOLUCAO'].apply(extrair_contexto_por_palavras)
dataframe_formatado.head(5)

,DS_EVOLUCAO,VALIDACAO,DS_CONTEXTO
0,ANOTACAO DE ENFERMAGEM - NOTURNO PACIENTE EVOL...,0,[]
1,EVOLUCAO DE FISIOTERAPIANOITE\n EM IH POR:- IT...,1,[STB PURULENTA EM TQT (VIA TRACH CARE) E POR V...
2,EVOLUCAO DE ENFERMAGEM - UTI CLINICA I - DIURN...,1,[EM VM; MANTEM SISTEMA DE ASPIRACAO FECHADO (T...
3,EVOLUCAO DE FISIOTERAPIAMANHA\n EM IH POR:- IT...,1,[STB PURULENTA EM TQT (VIA TRACH CARE) E POR V...
4,PLANTAO MTEM INTERNACAO NA UTI CLINICA I POR S...,0,[]


In [133]:

path = os.path.join('Dados_Tratados/')
file_name = 'Dados_Tratados_' + f"{datetime.today().strftime('%d-%m-%y')}" + ".xlsx"
file_path = path + file_name
dataframe_formatado.to_excel(file_path)